In [ ]:
df_test = pd.read_csv('train.csv')

# HomePlanet
df_test['HomePlanet'] = df_test['HomePlanet'].fillna(df_test['HomePlanet'].value_counts().index[0])
df_test['HomePlanet'] = df_test['HomePlanet'].astype('category')

# Luxuries
df_test['RoomService'].fillna(0.0, inplace = True)
df_test['FoodCourt'].fillna(0.0, inplace = True)
df_test['ShoppingMall'].fillna(0.0, inplace = True)
df_test['Spa'].fillna(0.0, inplace = True)
df_test['VRDeck'].fillna(0.0, inplace = True)

# CryoSleep
def is_cryo(a,b,c,d,e):
    if (a == 0.0) and (b == 0.0) and (c == 0.0) and (d == 0.0) and (e == 0.0):
        return True
    else:
        return False

df_nan_cryo = df_test[df_test['CryoSleep'].isnull()]
df_test = df_test.drop(df_nan_cryo.index)

df_nan_cryo['CryoSleep'] = df_nan_cryo.apply(lambda row: is_cryo(row['RoomService'], row['FoodCourt'], row['ShoppingMall'], row['Spa'], row['VRDeck']), axis = 1)

df_test = df_test.append(df_nan_cryo)
df_test = df_test.sort_index()

# Cabin
to_drop = df_test[df_test['Cabin'].isnull()].index
df_test.drop(to_drop, inplace = True)


df_test[['Cabin_deck', 'Cabin_num', 'Cabin_side' ]] = df_test.Cabin.str.split("/", expand = True)
df_test = df_test.drop(['Cabin'], axis = 1)

df_test['Cabin_deck'] = df_test['Cabin_deck'].replace(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], [1, 2, 3, 4, 5, 6, 7, 8])
df_test['Cabin_deck'] = df_test['Cabin_deck'].astype('category')

df_test['Cabin_num'] = df_test['Cabin_num'].astype('int')

df_test['Cabin_side'] = df_test['Cabin_side'].replace(['P', 'S'], [1,0])
df_test['Cabin_side'] = df_test['Cabin_side'].astype('bool')


# Destination
df_test['Destination'] = df_test['Destination'].astype('category')
df_test['Destination'] = df_test['Destination'].fillna(df_test['Destination'].value_counts().index[0])

# Age
df_test['Age'].fillna(df_test['Age'].mean(), inplace=True)
df_test['Age'] = df_test['Age'].astype('int')

# VIP
to_drop = df_test[df_test['VIP'].isnull()].index
df_test.drop(to_drop, inplace = True)
df_test['VIP'] = df_test['VIP'].astype('bool')

# Luxuries
amount_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df_test['Spendings'] = df_test[amount_columns].sum(axis = 1)
df_test = df_test.drop(amount_columns, axis = 1)

df_test = pd.get_dummies(df_test, columns = ['Cabin_deck', 'HomePlanet', 'Destination'])
df_test = df_test.drop(['PassengerId'], axis = 1)
target = df_test.pop('Transported')
df_test.insert(24, 'Transported', target)


In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.ensemble import RandomForestClassifier


for max_depth in range(7):
    for min_samples_split in range(4):
        for min_samples_leaf in range(3):
            clf = RandomForestClassifier(
            criterion='gini',
            n_estimators=200,
            max_depth=5,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features=None)

            kf=KFold(n_splits=5)
            score_cv=cross_val_score(clf,X,y,cv=kf)
            score = round(score_cv.mean(), 3)

            print('*********** Parameters ***********')
            print(f'Max Depth: {max_depth}')
            print(f'Min Samples Split: {min_samples_split}')
            print(f'Min Samples Leaf: {min_samples_leaf}')
            print (f"Average accuracy: {score}")